# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# import libraries
import re
import pickle
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xwen2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\xwen2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xwen2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')

df = pd.read_sql_table('InsertTableName', engine)

X = df['message']
Y = df.iloc[:,4:]

Y['related']=Y['related'].map(lambda x: 1 if x == 2 else x)

category_names = Y.columns

### 2. Write a tokenization function to process your text data

In [26]:
def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

sentence = 'I am a student, Vince!'
print(tokenize(sentence))

['student', 'vince']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [47]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    #('clf', MultiOutputClassifier(DecisionTreeClassifier(random_state = 42), n_jobs = -1))
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
pipeline.fit(X_train, y_train)

C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [49]:
y_pred = pipeline.predict(X_test)
print(classification_report(Y_test.values, y_pred, target_names=category_names))
print('Accuracy: {}'.format(np.mean(Y_test.values == y_pred)))

                        precision    recall  f1-score   support

               related       0.77      0.91      0.83      3993
               request       0.39      0.19      0.25       883
                 offer       0.00      0.00      0.00        24
           aid_related       0.44      0.37      0.40      2150
          medical_help       0.11      0.01      0.01       419
      medical_products       0.00      0.00      0.00       256
     search_and_rescue       0.00      0.00      0.00       138
              security       0.00      0.00      0.00       108
              military       0.14      0.01      0.01       171
           child_alone       0.00      0.00      0.00         0
                 water       0.07      0.00      0.01       316
                  food       0.21      0.04      0.06       564
               shelter       0.24      0.01      0.03       468
              clothing       0.00      0.00      0.00        85
                 money       0.00      

C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\metrics\_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [50]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x00000153B0B2DC80>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                           class_weight=None,
                                                                           dual=True,
                                                                           fit_intercept=True,
         

In [53]:
'''
parameters = {
    'vect__min_df': [1],
    'vect__lowercase': [False],
    'tfidf__smooth_idf': [False],
}

model = GridSearchCV(pipeline, param_grid = parameters, cv = 2)
'''

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))
                    ])

# hyper-parameter grid
parameters = {'vect__ngram_range': ((1, 1), (1, 2)),
              'vect__max_df': (0.75, 1.0)
             }

# create model
model = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=3, cv=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [54]:
model.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.118, total=  11.0s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.9s remaining:    0.0s
C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.123, total=  11.0s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


...

[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.131, total=  13.3s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2) ......................


C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.125, total=  12.9s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  2.4min finished
C:\Users\xwen2\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [55]:
model.best_score_

0.12811826112443334

### 9. Export your model as a pickle file

In [59]:
y_pred = model.predict(X_test)
print(classification_report(Y_test.values, y_pred, target_names=category_names))
print('Accuracy: {}'.format(np.mean(Y_test.values == y_pred)))


with open('clf.pickle', 'wb') as f:
    pickle.dump(model, f)

                        precision    recall  f1-score   support

               related       0.77      0.93      0.84      3993
               request       0.40      0.22      0.28       883
                 offer       0.00      0.00      0.00        24
           aid_related       0.45      0.37      0.41      2150
          medical_help       0.00      0.00      0.00       419
      medical_products       0.00      0.00      0.00       256
     search_and_rescue       0.00      0.00      0.00       138
              security       0.00      0.00      0.00       108
              military       0.00      0.00      0.00       171
           child_alone       0.00      0.00      0.00         0
                 water       0.12      0.00      0.01       316
                  food       0.26      0.04      0.07       564
               shelter       0.20      0.01      0.02       468
              clothing       0.00      0.00      0.00        85
                 money       0.00      

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.